In [3]:
"""
1️⃣ Basic Preprocessing (최종본)
----------------------------------------
EDA 전에 한 번만 실행하는 데이터 정리 단계입니다.

📌 주요 작업
1. 확장자 통일 (.jpg)
2. 손상 이미지 제거
3. 중복 이미지 제거 (hash 기반)
4. 해상도 통계 출력
5. train.csv 결합 → meta_cleaned.csv 저장
"""

import os, glob, hashlib
import cv2
import pandas as pd
from tqdm import tqdm


# ============================================================
# 🧩 1️⃣ 기본 유효성 검사 함수
# ============================================================
def is_valid_image(path):
    """이미지 파일이 손상되지 않았는지 검사"""
    try:
        img = cv2.imread(path)
        if img is None:
            return False
        h, w = img.shape[:2]
        return h > 0 and w > 0
    except Exception:
        return False


# ============================================================
# ⚙️ 2️⃣ 기본 전처리 메인 함수
# ============================================================
def basic_preprocessing(
    img_dir="../../data/raw/train",
    train_csv="../../data/raw/train.csv",
    save_meta_path="../../data/interim/meta_cleaned.csv",
    exts=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"),
    sample_n=100
):
    print(f"[SETUP] img_dir = {os.path.abspath(img_dir)}")
    if not os.path.isdir(img_dir):
        raise FileNotFoundError(f"이미지 폴더가 없습니다: {os.path.abspath(img_dir)}")

    os.makedirs(os.path.dirname(save_meta_path), exist_ok=True)
    print(f"[SETUP] save_meta_path = {os.path.abspath(save_meta_path)}")

    # --- 1. 이미지 파일 탐색 ---
    all_paths = []
    for ext in exts:
        all_paths += glob.glob(os.path.join(img_dir, "**", f"*{ext}"), recursive=True)
        all_paths += glob.glob(os.path.join(img_dir, "**", f"*{ext.upper()}"), recursive=True)
    all_paths = sorted(set(all_paths))
    print(f"🔧 총 {len(all_paths)}개 파일 후보 탐색 완료")

    # --- 2. 손상 이미지 제거 ---
    print("🧹 손상 이미지 검사 중...")
    valid_paths = [p for p in tqdm(all_paths) if is_valid_image(p)]
    print(f"   → 유효 이미지: {len(valid_paths)}")

    # --- 3. 중복 제거 (md5 해시) ---
    print("♻️ 중복 이미지 제거 중 (md5 해시)...")
    hashes = {}
    unique_paths = []
    for p in tqdm(valid_paths):
        try:
            with open(p, "rb") as f:
                h = hashlib.md5(f.read()).hexdigest()
            if h not in hashes:
                hashes[h] = p
                unique_paths.append(p)
        except Exception as e:
            print(f"⚠️ 해시 계산 실패: {p} ({e})")
    print(f"   → 중복 제거 후: {len(unique_paths)}")

    # --- 4. 해상도 통계 ---
    print("📏 해상도 통계 (샘플):")
    sizes = []
    for p in unique_paths[:min(sample_n, len(unique_paths))]:
        img = cv2.imread(p)
        if img is not None:
            h, w = img.shape[:2]
            sizes.append((w, h))
    if sizes:
        size_df = pd.DataFrame(sizes, columns=["width", "height"])
        print(size_df.describe())
    else:
        print("   (표본이 없어 통계를 출력할 수 없습니다)")

    # --- 5. train.csv 결합 ---
    print("📎 train.csv 결합 중...")
    meta = pd.read_csv(train_csv)

    # ID → filename 매칭
    if "ID" not in meta.columns:
        raise KeyError("train.csv에 'ID' 컬럼이 없습니다. 컬럼명을 확인하세요.")
    meta["filename"] = meta["ID"].astype(str)
    meta["filename"] = meta["filename"].apply(lambda x: x if x.endswith(".jpg") else f"{x}.jpg")

    # 라벨 컬럼 자동 탐지
    label_col = None
    for c in meta.columns:
        name = c.lower()
        if "label" in name or "target" in name or "class" in name:
            label_col = c
            break
    if label_col is None:
        raise KeyError("train.csv에서 label/target/class 컬럼을 찾을 수 없습니다.")
    else:
        print(f"✅ 라벨 컬럼 자동 감지: {label_col}")

    # 경로 매칭
    rel_paths = [os.path.relpath(p, img_dir).replace("\\", "/") for p in unique_paths]
    filenames = [os.path.basename(p) for p in unique_paths]
    out_df = pd.DataFrame({
        "filepath": rel_paths,
        "filename": filenames
    }).merge(meta[["filename", label_col]], on="filename", how="left")

    out_df.rename(columns={label_col: "target"}, inplace=True)
    out_df.to_csv(save_meta_path, index=False, encoding="utf-8-sig")
    print(f"💾 meta_cleaned.csv 저장 완료 → {save_meta_path}")

    # --- 로그 저장 ---
    log_path = os.path.join(os.path.dirname(save_meta_path), "preprocessing_log.txt")
    with open(log_path, "w", encoding="utf-8") as f:
        f.write(f"Total raw files: {len(all_paths)}\n")
        f.write(f"Valid images: {len(valid_paths)}\n")
        f.write(f"Unique images: {len(unique_paths)}\n")
        f.write(f"Meta entries: {len(meta)}\n")
        f.write(f"Cleaned entries: {len(out_df)}\n")
        f.write(f"Detected label column: {label_col}\n")
    print(f"🪵 로그 저장 완료 → {log_path}")

    print("✅ Basic Preprocessing 완료!")
    return out_df


# ============================================================
# 🚀 실행 섹션
# ============================================================
if __name__ == "__main__":
    IMG_DIR = "../../data/raw/train"
    TRAIN_CSV = "../../data/raw/train.csv"
    SAVE_META = "../../data/interim/meta_cleaned.csv"

    os.makedirs(os.path.dirname(SAVE_META), exist_ok=True)
    cleaned_df = basic_preprocessing(
        img_dir=IMG_DIR,
        train_csv=TRAIN_CSV,
        save_meta_path=SAVE_META
    )
    print("rows:", len(cleaned_df))


# ============================================================
# 🧭 2️⃣ 그룹 매핑 섹션
# ============================================================
META_PATH = "../../data/interim/meta_cleaned.csv"
META_MAP = "../../data/raw/meta.csv"   # ✅ target ↔ class_name 매핑표
IMG_DIR = "../../data/raw/train"
SAVE_DIR = "../reports/figures"

os.makedirs(SAVE_DIR, exist_ok=True)

# --- 1. 데이터 로드 ---
df = pd.read_csv(META_PATH)
print("✅ meta_cleaned.csv 로드 완료:", df.shape)
print(df.head())

# --- 2. class_name 병합 ---
meta_map = pd.read_csv(META_MAP)
df = df.merge(meta_map, on="target", how="left")
print("✅ class_name 컬럼 병합 완료:", df[["filepath", "target", "class_name"]].head())

# --- 3. 클래스 → 그룹 매핑 ---
class_to_group_mapping = {
    # 비정형
    'account_number': 'handwritten',

    # 의료계열
    'application_for_payment_of_pregnancy_medical_expenses': 'medical_cert',
    'confirmation_of_admission_and_discharge': 'medical_cert',
    'diagnosis': 'medical_cert',
    'prescription': 'medical_cert',
    'medical_outpatient_certificate': 'medical_cert',
    'statement_of_opinion': 'medical_cert',
    'payment_confirmation': 'medical_cert',

    'medical_bill_receipts': 'medical_receipt',
    'pharmaceutical_receipt': 'medical_receipt',

    # 신분증류
    'driver_lisence': 'identity',  # 오타 그대로
    'national_id_card': 'identity',
    'passport': 'identity',

    # 자동차 관련
    'car_dashboard': 'vehicle',
    'vehicle_registration_certificate': 'vehicle',
    'vehicle_registration_plate': 'vehicle',

    # 일반문서
    'resume': 'document',
}

group_to_idx = {
    'medical_cert': 0,
    'medical_receipt': 1,
    'identity': 2,
    'vehicle': 3,
    'document': 4,
    'handwritten': 5
}

# --- 4. 매핑 적용 ---
df["group"] = df["class_name"].map(class_to_group_mapping)
df["group_idx"] = df["group"].map(group_to_idx)

print("✅ 그룹 매핑 완료:")
print(df[["filepath", "class_name", "group", "group_idx"]].head())

# --- 5. 결과 저장 ---
df.to_csv("../../data/interim/meta_grouped.csv", index=False, encoding="utf-8-sig")
print("💾 meta_grouped.csv 저장 완료")


[SETUP] img_dir = /data/ephemeral/home/data/raw/train
[SETUP] save_meta_path = /data/ephemeral/home/data/interim/meta_cleaned.csv
🔧 총 1570개 파일 후보 탐색 완료
🧹 손상 이미지 검사 중...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1570/1570 [00:01<00:00, 935.03it/s]


   → 유효 이미지: 1570
♻️ 중복 이미지 제거 중 (md5 해시)...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1570/1570 [00:00<00:00, 8586.29it/s]


   → 중복 제거 후: 1570
📏 해상도 통계 (샘플):
            width      height
count  100.000000  100.000000
mean   495.030000  540.460000
std     77.727893   74.905519
min    443.000000  401.000000
25%    443.000000  443.000000
50%    443.000000  591.000000
75%    591.000000  591.000000
max    653.000000  591.000000
📎 train.csv 결합 중...
✅ 라벨 컬럼 자동 감지: target
💾 meta_cleaned.csv 저장 완료 → ../../data/interim/meta_cleaned.csv
🪵 로그 저장 완료 → ../../data/interim/preprocessing_log.txt
✅ Basic Preprocessing 완료!
rows: 1570
✅ meta_cleaned.csv 로드 완료: (1570, 3)
               filepath              filename  target
0  002f99746285dfdd.jpg  002f99746285dfdd.jpg      16
1  008ccd231e1fea5d.jpg  008ccd231e1fea5d.jpg      10
2  008f5911bfda7695.jpg  008f5911bfda7695.jpg      10
3  009235e4c9c07af5.jpg  009235e4c9c07af5.jpg       4
4  00b2f44967580c74.jpg  00b2f44967580c74.jpg      16
✅ class_name 컬럼 병합 완료:                filepath  target                  class_name
0  002f99746285dfdd.jpg      16  vehicle_registration_pla